<a href="https://colab.research.google.com/github/wan19990901/EECS6.S982_Sentiment/blob/sentiment_gwan/BigQuery_MIMIC_cookbook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

See more at: https://pandas-gbq.readthedocs.io/en/latest/intro.html#logging

In [ ]:
# Required files
import os
import sys
import pandas as pd
import pandas_gbq
import re

# Connect colab with google bigquery stuff
from google.colab import auth
from google.cloud import bigquery
from google.colab import files
from google.colab import drive

**Authenticate:** The line of code below ensures you are an authenticated user accessing the MIMIC database. You will need to rerun this each time you open the notebook.

In [ ]:
auth.authenticate_user()

**Query Function:** This is a method that executes a desired SQL query on the database. If you want to run a query, you can use the function name below, which we named run_query()

In [ ]:

project_id='rising-medley-364700' # Unique to each person
os.environ["GOOGLE_CLOUD_PROJECT"]=project_id

In [ ]:
# Your regular sql query

# For example:
query = '''
SELECT 
notes.ROW_ID,
icu.ICUSTAY_ID,
notes.TEXT,
hw.weight_first/((hw.height_first/100) * (hw.height_first/100)) as BMI

 FROM `physionet-data.mimiciii_notes.noteevents` as notes
 inner join `physionet-data.mimiciii_clinical.icustays` as icu
 on icu.ROW_ID = notes.ROW_ID

 inner join `physionet-data.mimiciii_derived.heightweight` as hw
 on hw.icustay_id = icu.ICUSTAY_ID

 where hw.weight_first is not null and hw.height_first is not null
 
'''

## Note that for we have to be careful on ID matching

In [ ]:
# Then directly query mimic data -- takes about a minute to run
heightreports = pandas_gbq.read_gbq(query, project_id=project_id, dialect = 'standard')


Downloading: 100%|██████████| 28892/28892 [06:57<00:00, 69.26rows/s]


In [ ]:
cutoff = 35
heightreports['obesity'] = 0
heightreports[heightreports.BMI > cutoff]['BMI'] = 1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
heightreports['BMI'] = heightreports['BMI'].astype('float')

In [ ]:
heightreports.to_csv('bmi.csv',index = False)

# Exported derived dataset back to BiqQuery for later analysis

In [ ]:

table_id = 'mimic_cohort.obesity'

pandas_gbq.to_gbq(heightreports, table_id, project_id=project_id) # using 35 as threshold


0it [00:00, ?it/s]
1it [01:32, 92.64s/it]


In [ ]:
query = '''
SELECT 
*
FROM mimic_cohort.obesity
'''
obesity = pandas_gbq.read_gbq(query, project_id=project_id, dialect = 'standard')

Downloading: 100%|██████████| 28892/28892 [02:07<00:00, 226.81rows/s]


In [ ]:
obesity['sentiment'] = 0

In [ ]:
obesity.loc[obesity.TEXT.str.lower().str.contains('very | pleasant'),'sentiment'] = 1

In [ ]:
obesity['obesity'] = 0
obesity.loc[obesity.BMI > 40,'obesity'] = 1

In [ ]:
obesity.groupby([
  'obesity','sentiment'
]).size().to_csv('obesity_count.csv')

In [ ]:
pd.read_csv('obesity_count.csv')

,obesity,sentiment,0
0,0,0,8663
1,0,1,18584
2,1,0,541
3,1,1,1104
